In [1]:
import pandas as pd
import numpy as np
import sys
import random
import os
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv', usecols=['lat', 'long', 'id'])
kampala_devices.head()

,id,lat,long
0,930434,0.360209,32.610756
1,718028,0.307500,32.620600
2,912224,0.346460,32.703280
3,930426,0.365500,32.646800
4,930427,0.268900,32.588000


In [4]:
kampala_df = pd.read_csv('../data/kampala_data.csv', parse_dates=['timestamp'])
kampala_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
1,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
2,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
3,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
4,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26


In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value', 'device_number']
# for i, device_id in enumerate(device_ids):
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5,device_number
0,453031.0,0.356989,32.613888,10.5477,930434
1,453032.0,0.356989,32.613888,16.4250,930434
2,453033.0,0.356989,32.613888,17.7239,930434
3,453034.0,0.356989,32.613888,16.1533,930434
4,453035.0,0.356989,32.613888,18.0123,930434


In [6]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(35, 35, 34)

In [7]:
final_df = final_df.drop(['device_number'], axis=1)
final_df.head()

,time,latitude,longitude,pm2_5
0,453031.0,0.356989,32.613888,10.5477
1,453032.0,0.356989,32.613888,16.4250
2,453033.0,0.356989,32.613888,17.7239
3,453034.0,0.356989,32.613888,16.1533
4,453035.0,0.356989,32.613888,18.0123


#### Model training and validation

In [8]:
def lstm(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.LSTM(50),
#     tf.keras.layers.Dense(1)
# ])
    
    input_layer = Input(shape=(X_train.shape[1],))
    reshaped_input = tf.expand_dims(input_layer, axis=1)
    
    lstm_layer1 = LSTM(units=50, return_sequences=True)(reshaped_input)
    dropout_layer1 = Dropout(dropout)(lstm_layer1)

    lstm_layer2 = LSTM(units=50)(dropout_layer1)
    dropout_layer2 = Dropout(dropout)(lstm_layer2)
    
    output_layer = Dense(units=1)(dropout_layer2)

    model = Model(inputs=input_layer, outputs=output_layer) 

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_split=0.2)
    
#     model = keras.Sequential()
#     model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(LSTM(50, return_sequences=True, input_shape=(window_size, x_train.shape[-1])))
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM((window_size * 2), return_sequences=True))) 
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM(window_size, return_sequences=False))) 
#     model.add(Dense(units=1))
    return model

In [9]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

In [10]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = lstm(X_train, y_train) #, dropout=0.1)
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [11]:
len(latitudes)

35

In [12]:
rmse_list, mape_list = [], []
# for i in range(len(latitudes[30:])):
lat_list = list(latitudes)
for lat in latitudes[8:12]:
    i = lat_list.index(lat)
#     print(i)
    try:
        rmse, mape = cross_validation(final_df, i)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'Location {i} successful')
    except Exception as e:
        print(e)
        print(f'Location {i} failed')
#         break

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
mean_rmse, mean_mape

Epoch 1/1000
1164/1164 [==============================] - 7s 4ms/step - loss: 660.9969 - val_loss: 605.6199
Epoch 2/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 482.8802 - val_loss: 605.2664
Epoch 3/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 483.5026 - val_loss: 604.9468
Epoch 4/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 483.0131 - val_loss: 604.2681
Epoch 5/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 483.0919 - val_loss: 604.9045
Epoch 6/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 483.8183 - val_loss: 605.6452
Epoch 7/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 482.4853 - val_loss: 605.8729
Epoch 8/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 482.8393 - val_loss: 604.6686
Epoch 9/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 483.2879 - val_loss: 605.7556
Epoch 10/1000
1164/1164 [===

1164/1164 [==============================] - 3s 3ms/step - loss: 482.2492 - val_loss: 604.8377
Epoch 77/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.8035 - val_loss: 605.4078
Epoch 78/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 482.6477 - val_loss: 603.8235
Epoch 79/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.9708 - val_loss: 604.2473
Epoch 80/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 482.5508 - val_loss: 607.7532
Epoch 81/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 482.5358 - val_loss: 604.8347
Epoch 82/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 482.2804 - val_loss: 604.4390
Epoch 83/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.8303 - val_loss: 605.6569
Epoch 84/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.9412 - val_loss: 603.5558
Epoch 85/1000
1164/1164 [========

Epoch 151/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 482.0511 - val_loss: 604.7745
Epoch 152/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.7180 - val_loss: 604.3952
Epoch 153/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.4707 - val_loss: 603.0621
Epoch 154/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.7339 - val_loss: 605.3572
Epoch 155/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.8033 - val_loss: 605.2193
Epoch 156/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.8226 - val_loss: 606.2004
Epoch 157/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.8730 - val_loss: 603.2507
Epoch 158/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.2827 - val_loss: 603.6842
Epoch 159/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.3776 - val_loss: 603.5668
Epoch 160/

1164/1164 [==============================] - 3s 3ms/step - loss: 480.4308 - val_loss: 603.0045
Epoch 226/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.3245 - val_loss: 603.0544
Epoch 227/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.9481 - val_loss: 603.3629
Epoch 228/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.1861 - val_loss: 602.9468
Epoch 229/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.2295 - val_loss: 603.4064
Epoch 230/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.9167 - val_loss: 603.3405
Epoch 231/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.9325 - val_loss: 603.0832
Epoch 232/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.5703 - val_loss: 603.1783
Epoch 233/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.6133 - val_loss: 603.8795
Epoch 234/1000
1164/1164 

1164/1164 [==============================] - 3s 3ms/step - loss: 481.0139 - val_loss: 603.1551
Epoch 300/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.7862 - val_loss: 603.0286
Epoch 301/1000
1164/1164 [==============================] - 4s 3ms/step - loss: 479.9748 - val_loss: 603.6414
Epoch 302/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.8693 - val_loss: 603.5294
Epoch 303/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.1930 - val_loss: 602.9131
Epoch 304/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.8387 - val_loss: 603.4418
Epoch 305/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.9534 - val_loss: 603.5330
Epoch 306/1000
1164/1164 [==============================] - 4s 3ms/step - loss: 480.9557 - val_loss: 602.9424
Epoch 307/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.5795 - val_loss: 603.0479
Epoch 308/1000
1164/1164 

...

1174/1174 [==============================] - 7s 6ms/step - loss: 536.1339 - val_loss: 600.2915
Epoch 770/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9384 - val_loss: 600.2778
Epoch 771/1000
1174/1174 [==============================] - 6s 6ms/step - loss: 535.8464 - val_loss: 600.5135
Epoch 772/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8356 - val_loss: 600.5295
Epoch 773/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8669 - val_loss: 600.5414
Epoch 774/1000
1174/1174 [==============================] - 6s 6ms/step - loss: 536.0590 - val_loss: 600.3038
Epoch 775/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 536.1263 - val_loss: 600.3940
Epoch 776/1000
1174/1174 [==============================] - 6s 6ms/step - loss: 535.8316 - val_loss: 600.3900
Epoch 777/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9330 - val_loss: 600.2659
Epoch 778/1000
1174/1174 

1174/1174 [==============================] - 7s 6ms/step - loss: 535.9797 - val_loss: 600.3781
Epoch 844/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8935 - val_loss: 600.4150
Epoch 845/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9185 - val_loss: 600.2732
Epoch 846/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8597 - val_loss: 600.3398
Epoch 847/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8046 - val_loss: 600.3214
Epoch 848/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9831 - val_loss: 600.2663
Epoch 849/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9242 - val_loss: 600.4122
Epoch 850/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9198 - val_loss: 600.3623
Epoch 851/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.7350 - val_loss: 600.3813
Epoch 852/1000
1174/1174 

1174/1174 [==============================] - 7s 6ms/step - loss: 535.8812 - val_loss: 600.3112
Epoch 918/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8569 - val_loss: 600.2722
Epoch 919/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8918 - val_loss: 600.2588
Epoch 920/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9316 - val_loss: 600.3392
Epoch 921/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9247 - val_loss: 600.2586
Epoch 922/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8661 - val_loss: 600.4341
Epoch 923/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9218 - val_loss: 600.4192
Epoch 924/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9406 - val_loss: 600.2620
Epoch 925/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8433 - val_loss: 600.2576
Epoch 926/1000
1174/1174 

1174/1174 [==============================] - 7s 6ms/step - loss: 535.8569 - val_loss: 600.2638
Epoch 992/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9938 - val_loss: 600.4308
Epoch 993/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8541 - val_loss: 600.3018
Epoch 994/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9128 - val_loss: 600.2576
Epoch 995/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 536.0049 - val_loss: 600.2618
Epoch 996/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9528 - val_loss: 600.2767
Epoch 997/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8080 - val_loss: 600.2652
Epoch 998/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.8619 - val_loss: 600.2702
Epoch 999/1000
1174/1174 [==============================] - 7s 6ms/step - loss: 535.9317 - val_loss: 600.3657
Epoch 1000/1000
37/37 [==

(27.278519919895945, 0.626274751477196)

In [13]:
rmse_list

[41.42548680014174, 23.99175056321661, 26.329341275725305, 17.367501040500134]

In [14]:
mape_list

[0.45047949706207324,
 0.4481338172626773,
 0.6041677323129314,
 1.002317959271102]

In [16]:
total_rmse_list= [17.16103245591475, 
18.236217488130794, 
16.029602696364044, 
16.917543034802137, 
26.719072396228054,
 29.125487809358955, 
20.80745310377469, 
18.112215583488428,
41.42548680014174, 
23.99175056321661, 
26.329341275725305, 
17.367501040500134,
22.17414251495171, 
44.8827239104135, 
18.820082750201472, 
23.518873800012834, 
26.027613501527192, 
19.934448479318306, 
17.41480267797978, 
15.888245224620006,
36.29799613719305, 
17.795022591319018, 
17.530896061890893, 
16.4286055328919,
15.899894953399492, 
21.939405171876388, 
29.8501836189287, 
23.30797315133057,
18.29485720825161, 
17.715616332320828, 
18.371943097769513, 
29.618986440637478,
18.36592087472948, 
34.15487994437615]
np.mean(total_rmse_list)

22.836935830105453

In [17]:
total_mape_list = [0.6657039681666103,
 0.9797744432240922,
 0.9396034325051472,
 0.8040127137442739,
0.4491502837566655,
 0.5267754188547462,
 0.7004775438899236,
 0.7305040897516272,
0.45047949706207324,
 0.4481338172626773,
 0.6041677323129314,
 1.002317959271102,
0.6875690364798261,
 0.4639710381227825,
 1.4420136631826825,
 0.5447535233140856,
0.4337727949974403, 
0.5485902346042193, 
0.7687537803199184, 
0.8613226318251865,
0.424843706136675, 
0.9588746942306431, 
0.8819278798775211, 
0.7445293682109514,
0.6779708849831162,
 0.4447500325490989,
 0.4973972202085524,
 0.4142900151885426,
0.7876238252437539,
 1.0865796515153845,
 0.6912159160796465,
 0.4654280374577772,
0.6055699101549334, 
0.5449686129298168]
np.mean(total_mape_list)

0.6846416869827772